In [2]:
import pdfplumber, re
import pandas as pd

In [3]:
file_path = 'dataset.pdf'

l = ['P/L', 'P', 'TL', 'TH']
l1 = '|'.join(l)

In [5]:
# custom function for read data from pdf

def get_tabel(data):
    no_peserta, nama, pendidikan, tahun_skd, twk, tiu, tkp, total, keterangan = [], [], [], [], [], [], [], [], []
    for i in data:
        for j in i:
            if set(j).intersection(set(l)):
                no_peserta.append(j[1])
                nama.append(j[2])
                pendidikan.append(j[3])
                tahun_skd.append(j[4])
                twk.append(j[5])
                tiu.append(j[6])
                tkp.append(j[7])
                total.append(j[8])
                keterangan.append(j[9])
    if no_peserta:            
        df = pd.DataFrame({'no_peserta': no_peserta, 'nama': nama, 'pendidikan': pendidikan, 'tahun_skd': tahun_skd, 
                          'twk': twk, 'tiu': tiu, 'tkp': tkp, 'total': total, 'keterangan': keterangan})
        return df
    else: return None

        
def get_lokasi(data):
    data_atas = []
    data_tengah = []
    data_bawah = []
    idx_atas, idx_tengah, idx_bawah = 0, 0, 0
    
    for i, j in enumerate(data):
        if 'Jabatan' in j:
            idx_atas = i
        if 'Lokasi' in j:
            idx_tengah = i
        if 'Jenis Formasi' in j:
            idx_bawah = i
    
    if list(range(idx_atas+1, idx_tengah)): # kalo ada beberapa baris
        for i in list(range(idx_atas+1, idx_tengah)):
            data_atas.append(data[i])
    else: # perfect structure
        pass
    if list(range(idx_tengah+1, idx_bawah)): # kalo ada beberapa baris
        for i in list(range(idx_tengah+1, idx_bawah)):
            # print(data[i])
            data_bawah.append(data[i])
    if idx_tengah != 0:
        data_tengah.append(data[idx_tengah])
    d1 = " ".join(data_atas)
    d2 = " ".join(data_tengah)
    d3 = " ".join(data_bawah)
    return " ".join([d1, d2, d3])    


def get_instansi(data):
    for i in data:
        if 'Instansi' in i:
            return i


def get_jabatan(data):
    data_atas = []
    data_tengah = []
    data_bawah = []
    idx_atas, idx_tengah, idx_bawah = 0, 0, 0
    
    for i, j in enumerate(data):
        if 'Instansi' in j:
            idx_atas = i
        if 'Jabatan' in j:
            idx_tengah = i
        if 'Lokasi Formasi' in j:
            idx_bawah = i
    
    if list(range(idx_atas+1, idx_tengah)): # kalo ada beberapa baris
        for i in list(range(idx_atas+1, idx_tengah)):
            if data[i].isupper():
                data_atas.append(data[i])
    else: # perfect structure
        pass
    if list(range(idx_tengah+1, idx_bawah)): # kalo ada beberapa baris
        for i in list(range(idx_tengah+1, idx_bawah)):
            if data[i].isupper():
                data_bawah.append(data[i])
    if idx_tengah != 0:
        data_tengah.append(data[idx_tengah])
    d1 = " ".join(data_atas)
    d2 = " ".join(data_tengah)
    d3 = " ".join(data_bawah)
    return " ".join([d2, d1, d3])    


def get_jenis(data):
    for i in data:
        if 'Jenis' in i:
            return i


def get_pendidikan(data):
    for i in data:
        for j in i:
            if 'Pendidikan' in j and 'No' not in j:
                return j[1]

In [36]:
# process reading pdf and save it to dataframe

pdf = pdfplumber.open(file_path)

z = []
b1 = {
    'instansi': None, 'jabatan': None, 'lokasi': None, 'jenis': None, 'pendidikan_formasi': None
}
b2 = {
    'instansi': None, 'jabatan': None, 'lokasi': None, 'jenis': None, 'pendidikan_formasi': None
}
for i in range(len(pdf.pages)):
    page = pdf.pages[i]
    text = page.extract_tables()
    text1 = page.extract_text()
    text1 = text1.split('\n')
    b1['lokasi'] = get_lokasi(text1)
    b1['instansi'] = get_instansi(text1)
    b1['jabatan'] = get_jabatan(text1)
    b1['jenis'] = get_jenis(text1)
    b1['pendidikan_formasi'] = get_pendidikan(text)
    if b1['lokasi'] != '  ':
        b2['instansi'] = b1['instansi']
        b2['jabatan'] = b1['jabatan']
        b2['lokasi'] = b1['lokasi']
        b2['jenis'] = b1['jenis']
        b2['pendidikan_formasi'] = b1['pendidikan_formasi']
    a = get_tabel(text)
    if isinstance(a, pd.DataFrame):
        a['instansi_formasi'] = b2['instansi']
        a['jabatan_formasi'] = b2['jabatan']
        a['lokasi_formasi'] = b2['lokasi']
        a['jenis_formasi'] = b2['jenis']
        a['pendidikan_formasi'] = b2['pendidikan_formasi']
        
        z.append(a)

pdf.close()

df = pd.concat(z)

,no_peserta,nama,pendidikan,tahun_skd,twk,tiu,tkp,total,keterangan,instansi_formasi,jabatan_formasi,lokasi_formasi,jenis_formasi,pendidikan_formasi
0,24610120110002684,TEUKU SUHAIMI,S-1 HUKUM,2024,85,115,183,383,P/L,Instansi : 6101 Pemerintah Kab. Bogor 379,Jabatan Formasi : JF0000334 - ANALIS HUKUM AHL...,Pemerintah Kabupaten Bogor | Badan Pengelolaan...,Jenis Formasi : 1 - UMUM 1,(5123000) S-1 HUKUM
1,24610120120004288,INDAH LARASATI,S-1 HUKUM,2024,85,130,167,382,P/L,Instansi : 6101 Pemerintah Kab. Bogor 379,Jabatan Formasi : JF0000334 - ANALIS HUKUM AHL...,Pemerintah Kabupaten Bogor | Badan Pengelolaan...,Jenis Formasi : 1 - UMUM 1,(5123000) S-1 HUKUM
2,24610120120004289,SHINTA DEWANTI SETYANINGRUM,S-1 HUKUM,2024,70,100,183,353,P/L,Instansi : 6101 Pemerintah Kab. Bogor 379,Jabatan Formasi : JF0000334 - ANALIS HUKUM AHL...,Pemerintah Kabupaten Bogor | Badan Pengelolaan...,Jenis Formasi : 1 - UMUM 1,(5123000) S-1 HUKUM
3,24610120110002691,GEN YAISH IBRAHIM,S-1 HUKUM,2024,80,80,174,334,P,Instansi : 6101 Pemerintah Kab. Bogor 379,Jabatan Formasi : JF0000334 - ANALIS HUKUM AHL...,Pemerintah Kabupaten Bogor | Badan Pengelolaan...,Jenis Formasi : 1 - UMUM 1,(5123000) S-1 HUKUM
4,24610120120001034,GHAIDA AMBARWATI,S-1 HUKUM,2024,95,70,187,352,TL,Instansi : 6101 Pemerintah Kab. Bogor 379,Jabatan Formasi : JF0000334 - ANALIS HUKUM AHL...,Pemerintah Kabupaten Bogor | Badan Pengelolaan...,Jenis Formasi : 1 - UMUM 1,(5123000) S-1 HUKUM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,24610120120004635,LINDA MARLINA,D-III ANALIS\nKESEHATAN,2024,75,125,149,349,TL,Instansi : 6101 Pemerintah Kab. Bogor 379,Jabatan Formasi : JP4291395 - PENGELOLA LAYANA...,Pemerintah Kabupaten Bogor | Rumah Sakit Umum ...,Jenis Formasi : 1 - UMUM 1,(4411011) D-III ANALIS KESEHATAN/ (4431000) D-...
4,24610120120004672,NANDA RIZQI FIRDAUS,D-III FARMASI,2024,50,95,177,322,TL,Instansi : 6101 Pemerintah Kab. Bogor 379,Jabatan Formasi : JP4291395 - PENGELOLA LAYANA...,Pemerintah Kabupaten Bogor | Rumah Sakit Umum ...,Jenis Formasi : 1 - UMUM 1,(4411011) D-III ANALIS KESEHATAN/ (4431000) D-...
5,24610120120004669,KARTIKA LAKSANA,D-III ANALIS\nKESEHATAN,2024,35,100,155,290,TL,Instansi : 6101 Pemerintah Kab. Bogor 379,Jabatan Formasi : JP4291395 - PENGELOLA LAYANA...,Pemerintah Kabupaten Bogor | Rumah Sakit Umum ...,Jenis Formasi : 1 - UMUM 1,(4411011) D-III ANALIS KESEHATAN/ (4431000) D-...
6,24610120120004670,NANDA AULIA,D-III ANALIS\nKESEHATAN,2024,null,null,null,null,TH,Instansi : 6101 Pemerintah Kab. Bogor 379,Jabatan Formasi : JP4291395 - PENGELOLA LAYANA...,Pemerintah Kabupaten Bogor | Rumah Sakit Umum ...,Jenis Formasi : 1 - UMUM 1,(4411011) D-III ANALIS KESEHATAN/ (4431000) D-...


In [41]:
df.to_pickle('./dirty_table.pkl')